<a href="https://colab.research.google.com/github/thingumajig/colab-experiments/blob/master/GPT_2_abstractive_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# From GPT-2 Paper:
https://d4mucfpksywv.cloudfront.net/better-language-models/language-models.pdf

**3.6. Summarization**

We test GPT-2’s ability to perform summarization on the
CNN and Daily Mail dataset (Nallapati et al., 2016). To induce summarization behavior we add the text TL;DR: after
the article and generate 100 tokens with Top-k random sampling (Fan et al., 2018) with k = 2 which reduces repetition
and encourages more abstractive summaries than greedy decoding. We use the first 3 generated sentences in these 100
tokens as the summary. While qualitatively the generations
resemble summaries, as shown in Table 14, they often focus
on recent content from the article or confuse specific details
such as how many cars were involved in a crash or whether
a logo was on a hat or shirt. On the commonly reported
ROUGE 1,2,L metrics the generated summaries only begin
to approach the performance of classic neural baselines and
just barely outperforms selecting 3 random sentences from
the article. GPT-2’s performance drops by 6.4 points on
the aggregate metric when the task hint is removed which
demonstrates the ability to invoke task specific behavior in
a language model with natural language.





# .init

In [1]:
!git clone https://github.com/ilopezfr/gpt-2/
import os, sys
#os.chdir('gpt-2')
sys.path.insert(0, 'gpt-2')

!python gpt-2/download_model.py 117M
!python gpt-2/download_model.py 345M


Cloning into 'gpt-2'...
remote: Enumerating objects: 256, done.
remote: Total 256 (delta 0), reused 0 (delta 0), pack-reused 256
Receiving objects: 100% (256/256), 4.57 MiB | 17.03 MiB/s, done.
Resolving deltas: 100% (141/141), done.
Fetching checkpoint: 1.00kit [00:00, 867kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 35.3Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 911kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:10, 49.4Mit/s]                                  
Fetching model.ckpt.index: 6.00kit [00:00, 3.93Mit/s]                                               
Fetching model.ckpt.meta: 472kit [00:00, 34.0Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 33.3Mit/s]                                                       
Fetching checkpoint: 1.00kit [00:00, 871kit/s]             

# Code

In [2]:
!pip3 install -r gpt-2/requirements.txt
# sys.path.insert(0, 'gpt-2')
sys.path.insert(0, 'gpt-2/src')

import os

import json
import numpy as np
import tensorflow as tf

# from src import model, sample, encoder
import model, sample, encoder

import textwrap

def print_wrapped_text(raw_text):
  wrapper = textwrap.TextWrapper(width = 80)
  wrap_list = wrapper.wrap(text=raw_text)

  for line in wrap_list:
     print(line)


def generate_summary(
    raw_text,
    model_name='117M',
    seed=None,
    nsamples=1,
    batch_size=1,
    length=None,
    temperature=1,
    top_k=0,
    models_dir='models',
):
    """
    Interactively run the model
    :model_name=117M : String, which model to use
    :seed=None : Integer seed for random number generators, fix seed to reproduce
     results
    :nsamples=1 : Number of samples to return total
    :batch_size=1 : Number of batches (only affects speed/memory).  Must divide nsamples.
    :length=None : Number of tokens in generated text, if None (default), is
     determined by model hyperparameters
    :temperature=1 : Float value controlling randomness in boltzmann
     distribution. Lower temperature results in less random completions. As the
     temperature approaches zero, the model will become deterministic and
     repetitive. Higher temperature results in more random completions.
    :top_k=0 : Integer value controlling diversity. 1 means only 1 word is
     considered for each step (token), resulting in deterministic completions,
     while 40 means 40 words are considered at each step. 0 (default) is a
     special setting meaning no restrictions. 40 generally is a good value.
     :models_dir : path to parent folder containing model subfolders
     (i.e. contains the <model_name> folder)     
    """
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = encoder.get_encoder(model_name, models_dir)
    hparams = model.default_hparams()
    with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=context,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)

        context_tokens = enc.encode(raw_text)
        generated = 0
        for _ in range(nsamples // batch_size):
            out = sess.run(output, feed_dict={
                context: [context_tokens for _ in range(batch_size)]
            })[:, len(context_tokens):]
            for i in range(batch_size):
                generated += 1
                text = enc.decode(out[i])
                print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                print_wrapped_text(text)
        print("=" * 80)




     |████████████████████████████████| 51kB 3.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/2a/1a/4d/6b30377c3051e76559d1185c1dbbfff15aed31f87acdd14c22
Successfully built fire
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


# Abstractive summarization

In [6]:
#@title Parameters:
raw_text = "Describe any lessons learned during the various phases of the project. Consider lessons both for future projects, as well as for CEPF\u2019s future performance. Project Design Process: (aspects of the project design that contributed to its success/failure) \u2022 The project was well designed, drawing strongly on BirdLife\u2019s experience in strategic planning and capacity development of NGOs. \u2022 Retaining an output on project management and technical support was helpful (output 5) to keep track of the secretariat and Africa Partnership\u2019s role. \u2022 The decision to appoint key staff from the previous BirdLife (Projet ZICOMA) team, but then recruit additional staff openly, was successful in ensuring a solid BirdLife \u201Ccore\u201D in the team, but allowing new skills to be added. \u2022 The decision not to have a permanent expatriate technical adviser was successful in ensuring national ownership and compatibility with Asity (national NGO); at no time was this ever questioned. However, more time should have been budgeted for headquarters staff time (country programme officer): 50-75% (not one month, or <10%) of the time of a headquarters staff member would have been justified, and still much cheaper than an permanent adviser. Project Execution: (aspects of the project execution that contributed to its success/failure) \u2022 For training, internships, BirdLife network events and on-the-job training were preferred over training courses delivered by external agencies. This proved economical and generally successful. \u2022 Involvement of the BirdLife Africa Partnership was critical to the NGO development objectives, because the Partners (rather than secretariat) are in the best position to demonstrate the value of joining the network, and also because the Partnership itself determines whether or not a Malagasy organization will be admitted. This took place in two main ways. The chair of the Council for the African Partnership (Kinfe Abebe of the Ethiopian Wildlife and Natural History Society) attended the BIMP-Asity collaboration workshop in June 2003, and was able to explain at first hand the benefits to national NGOs of being a BirdLife Partner. The president of Asity (together with the director of BIMP) attended the Africa Partnership meeting in September, meeting representatives of all the Partners, and participating in the regional planning process. \u2022 Involvement of the BirdLife African Partnership secretariat in Nairobi (that is, the Africa Division of the BirdLife secretariat, as distinct from the Partnership itself, as described above) was also critical to NGO development objectives, because the staff have unique experience in facilitating the development and cohesion of national NGOs and networks. Key staff directly involved through visits to Madagascar and hosting internships in Nairobi were Hazell Thompson (head) and Maaike Manten (institutional fundraiser for Africa). \u2022 Close cooperation and frequent communication between the BIMP team (and latterly also Asity) and the secretariat headquarters helped to ensure success. Attention to detail and rapid correspondence were key parts of this, and so it was crucial that the country   8 programme officer (project manager) in UK was able allocate the necessary time. This work entails considerable UK-based costs and may be seen as administration, making it hard to fund. However, as this report makes clear, the support given is in fact largely technical, and the country programme officer system provides an economical way to provide the support needed. \u2022 The advice and support from CI as a like-minded NGO with a strong presence in Madagascar, and from CEPF as a donor with an exceptional level of commitment to seeing projects succeed, were essential to overall success. In addition to CEPF staff, whose contribution to the success of the project was greatly appreciated, Olivier Langrand (CI \u2013 Washington DC) provided crucial advice in the design of the project, and Frank Hawkins (CI \u2013 Madagascar) advised throughout, often by giving guidance directly to the BIMP team. " #@param {type:"string"}
text_length = 100 #@param {type:"slider", min:1, max:300, step:1}
num_samples = 3 #@param {type:"slider", min:1, max:10, step:1}
temperature = 1 #@param {type:"slider", min:0, max:1, step:0.1}
model_name = "345M" #@param ["345M", "117M"]
seed = 32 #@param {type:"integer"}
top_k = 2 #@param {type:"slider", min:0, max:100, step:1}
if seed == -1:
  seed = None

raw_text += '\nTL;DR:'
#raw_text += '\nSummary:'

print("=" * 30 + f" Source text: len={len(raw_text)} " + "=" * 30)    
print_wrapped_text(raw_text)
print('='*80)

# os.chdir('/content/gpt-2/')    
generate_summary(raw_text, model_name=model_name, 
                 nsamples=num_samples, 
                 length=text_length, 
                 temperature=temperature,
                 seed = seed,
                 top_k = top_k,
                 models_dir='/content/models/')


============================== Source text: len=4052 ==============================
Describe any lessons learned during the various phases of the project. Consider
lessons both for future projects, as well as for CEPF’s future performance.
Project Design Process: (aspects of the project design that contributed to its
success/failure) • The project was well designed, drawing strongly on BirdLife’s
experience in strategic planning and capacity development of NGOs. • Retaining
an output on project management and technical support was helpful (output 5) to
keep track of the secretariat and Africa Partnership’s role. • The decision to
appoint key staff from the previous BirdLife (Projet ZICOMA) team, but then
recruit additional staff openly, was successful in ensuring a solid BirdLife
“core” in the team, but allowing new skills to be added. • The decision not to
have a permanent expatriate technical adviser was successful in ensuring
national ownership and compatibility with Asity (national